In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

def add_picture_and_predict():
    from tensorflow.keras.preprocessing import image
    import numpy as np
    import tkinter as tk
    from tkinter import filedialog

    
    img_width, img_height = 150, 150

    
    print("Please upload an image (max 5MB) for gender prediction:")
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    image_path = filedialog.askopenfilename(title="Select Image File", filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])

    if not image_path:
        print("No file selected. Exiting...")
        return

    
    file_size = os.path.getsize(image_path) / (1024 * 1024)  # Size in MB
    if file_size > 5:
        print("File size exceeds 5MB. Please upload a smaller image.")
        return

    
    img = image.load_img(image_path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img) / 255.0
    img_array = tf.expand_dims(img_array, 0)  # Add batch dimension

   
    model = tf.keras.models.load_model("gender_classification_model.h5")

    
    prediction = model.predict(img_array)
    predicted_label = "Male" if prediction[0][0] > 0.5 else "Female"
    print(f"Predicted gender: {predicted_label}")


img_width, img_height = 150, 150
batch_size = 32
epochs = 10
data_dir = "dataset"  

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary",
    subset="training",
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary",
    subset="validation",
)


model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(img_width, img_height, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid"),
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
)


model.save("gender_classification_model.h5")
print("Model saved as gender_classification_model.h5")



FileNotFoundError: [WinError 3] The system cannot find the path specified: 'dataset'